In [1]:
from recovar import RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
from kfold_trainer import KfoldTrainer
from config import KFOLD_SPLITS

2025-07-14 11:23:56.946813: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 11:23:56.948854: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-14 11:23:56.976480: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-14 11:23:56.976509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-14 11:23:56.976525: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
# Experiment name.
EXP_NAME = "continuous_train"

# Should be one of the RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
MODEL_CLASSES = [RepresentationLearningSingleAutoencoder]#[RepresentationLearningMultipleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder]

# Should be stead or instance.
DATASETS = ["continuous"]

# Number of epochs
NUM_EPOCHS = 5

In [3]:
# For all splits, train the model over defined datasets.
for train_dataset in DATASETS:
    print(train_dataset)
    for model_class in MODEL_CLASSES:
        print(model_class)
        for split in range(KFOLD_SPLITS):
            print(split)
            kfold_trainer = KfoldTrainer(
                EXP_NAME, model_class, train_dataset, split, epochs=NUM_EPOCHS
            )
            print(kfold_trainer)
            kfold_trainer.train()

continuous
<class 'recovar.representation_learning_models.RepresentationLearningSingleAutoencoder'>
0


2025-07-14 11:24:07.723337: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/5
286/492 [================>.............] - ETA: 1:03 - loss: 1.0648

KeyboardInterrupt: 

In [1]:
from directory import CONTINUOUS_WAVEFORMS_HDF5_PATH, CONTINUOUS_METADATA_CSV_PATH
import os
import h5py
import pandas as pd

print("=== Debug Continuous Data ===")
print(f"HDF5 path: {CONTINUOUS_WAVEFORMS_HDF5_PATH}")
print(f"HDF5 exists: {os.path.exists(CONTINUOUS_WAVEFORMS_HDF5_PATH)}")

print(f"CSV path: {CONTINUOUS_METADATA_CSV_PATH}")
print(f"CSV exists: {os.path.exists(CONTINUOUS_METADATA_CSV_PATH)}")

if os.path.exists(CONTINUOUS_METADATA_CSV_PATH):
    df = pd.read_csv(CONTINUOUS_METADATA_CSV_PATH)
    print(f"CSV rows: {len(df)}")
    print(f"CSV columns: {list(df.columns)}")
    print(f"Labels: {df['label'].value_counts()}")

if os.path.exists(CONTINUOUS_WAVEFORMS_HDF5_PATH):
    with h5py.File(CONTINUOUS_WAVEFORMS_HDF5_PATH, 'r') as f:
        if 'data' in f:
            print(f"HDF5 traces: {len(f['data'].keys())}")
            print(f"Sample trace names: {list(f['data'].keys())[:3]}")
        else:
            print("No 'data' group in HDF5 file")

=== Debug Continuous Data ===
HDF5 path: /home/ege/recovar/reproducibility/output/by_station/SLVT_continuous_waveforms.hdf5
HDF5 exists: True
CSV path: /home/ege/recovar/reproducibility/output/by_station/SLVT_continuous_metadata.csv
CSV exists: True
CSV rows: 210279
CSV columns: ['trace_name', 'station_name', 'network', 'trace_start_time', 'label', 'p_arrival_sample', 's_arrival_sample', 'source_id', 'trace_category']
Labels: label
no    209809
eq       470
Name: count, dtype: int64
HDF5 traces: 210279
Sample trace names: ['KO.SLVT.20190907_022132', 'KO.SLVT.20190907_022202', 'KO.SLVT.20190907_022332']


In [2]:
from kfold_environment import KFoldEnvironment

# Create environment and get generators
env = KFoldEnvironment("continuous")
train_gen, val_gen, test_gen, predict_gen = env.get_generators(0)

print(f"Test generator length: {len(test_gen)}")
print(f"Predict generator length: {len(predict_gen)}")

# Try to get the first batch
if len(predict_gen) > 0:
    try:
        x = predict_gen[0]
        print(f"First batch type: {type(x)}")
        print(f"First batch shape: {x.shape if x is not None else 'None'}")
        
        # Also try the test generator
        x_test, y_test = test_gen[0]
        print(f"Test batch x shape: {x_test.shape if x_test is not None else 'None'}")
        print(f"Test batch y shape: {y_test.shape if y_test is not None else 'None'}")
        
    except Exception as e:
        print(f"Error getting batch: {e}")
        import traceback
        traceback.print_exc()
        
        # Let's check the chunk metadata
        print(f"\nFirst chunk size: {len(env.chunk_metadata_list[0])}")
        print(f"First chunk sample:")
        print(env.chunk_metadata_list[0].head(1))

2025-07-14 11:22:10.312189: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 11:22:10.314373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-14 11:22:10.339117: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-14 11:22:10.339135: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-14 11:22:10.339151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Test generator length: 164
Predict generator length: 164
First batch type: <class 'NoneType'>
First batch shape: None
Test batch x shape: None
Test batch y shape: None


In [3]:
from kfold_environment import KFoldEnvironment
from directory import PREPROCESSED_DATASET_DIRECTORY
import os

env = KFoldEnvironment("continuous")

# Check the preprocessed file path
processed_hdf5_path = os.path.join(
    PREPROCESSED_DATASET_DIRECTORY,
    "continuous",
    "subsampled_100percent.hdf5"
)

print(f"Preprocessed HDF5 path: {processed_hdf5_path}")
print(f"Preprocessed HDF5 exists: {os.path.exists(processed_hdf5_path)}")

# Check if the directory exists
processed_dir = os.path.join(PREPROCESSED_DATASET_DIRECTORY, "continuous")
print(f"Preprocessed directory: {processed_dir}")
print(f"Preprocessed directory exists: {os.path.exists(processed_dir)}")

if os.path.exists(processed_dir):
    print(f"Files in directory: {os.listdir(processed_dir)}")

Preprocessed HDF5 path: /home/ege/recovar_data_preprocessed/continuous/subsampled_100percent.hdf5
Preprocessed HDF5 exists: True
Preprocessed directory: /home/ege/recovar_data_preprocessed/continuous
Preprocessed directory exists: True
Files in directory: ['metadata.csv', 'subsampled_100percent.hdf5']


In [5]:
import h5py
import numpy as np

preprocessed_path = "/home/ege/recovar_data_preprocessed/continuous/subsampled_100percent.hdf5"

with h5py.File(preprocessed_path, 'r') as f:
    print("Groups in preprocessed file:", list(f.keys()))
    
    if 'data' in f:
        print("Groups in data:", list(f['data'].keys()))
        
        if 'x' in f['data']:
            print("Groups in data/x:", list(f['data']['x'].keys()))
            if 'chunk0' in f['data']['x']:
                print("Batches in chunk0:", list(f['data']['x']['chunk0'].keys()))
                
                # Try to read the first batch
                if '0' in f['data']['x']['chunk0']:
                    batch_x = f['data']['x']['chunk0']['0'][:]
                    print(f"First batch x shape: {batch_x.shape}")
                    print(f"First batch x type: {batch_x.dtype}")
                    print(f"First batch x sample: {batch_x[0, :5, 0] if len(batch_x.shape) > 2 else 'Wrong shape'}")
                else:
                    print("No batch '0' in chunk0")
            else:
                print("No 'chunk0' in data/x")
        else:
            print("No 'x' in data")
            
        if 'y' in f['data']:
            print("Groups in data/y:", list(f['data']['y'].keys()))
    else:
        print("No 'data' group in preprocessed file")

OSError: Unable to synchronously open file (file locking 'ignore disabled locks' flag values don't match)